In [1]:
from glob import glob
from utils import to_markdown
from urllib.parse import urlparse
from collections import defaultdict

In [2]:
def contains_course_info(url):
    url_prased = urlparse(url)
    return len(url_prased.path.split("/"))>5

In [3]:
files = []
with open("stevens_scraper/crawled_urls.txt") as fp:
    for line in fp:
        files.append(line.strip())
files = list(map(lambda x: x.lower(), files))
files = list(set(files))
files = list(filter(lambda file: "narrative-courses" not in file, files))
files = list(filter(lambda file: "javascript" not in file, files))
files = list(filter(lambda file: ".pdf" not in file, files))
files = list(filter(lambda file: ".aspx" not in file, files))
files = list(filter(lambda file: "degree-programs2" not in file, files))
files = list(filter(lambda file: "2023-2024" in file, files))
files = list(filter(lambda file: "#middle" not in file, files))
files = list(filter(contains_course_info , files))
print(len(files))

with open("stevens_scraper/crawled_urls_v1.txt", "w+") as fp:
    fp.write("\n".join(sorted(files, key=len)))

3434


In [4]:
primary = defaultdict(list)
for url in files:
    url_prased = urlparse(url.lower())
    cat = url_prased.path.replace("/en/2023-2024/academic-catalog/", "").split("/")[0]
    primary[cat].append(url)

In [5]:
for key in primary.keys(): print(key, len(primary[key]))

courses 2889
student-services 26
department-of-computer-science 23
tuition-fees-and-other-expenses-for-graduate-students 5
school-of-business 73
school-of-systems-and-enterprises 41
 12
financing-a-stevens-education 44
undergraduate-education 77
department-of-civil-environmental-and-ocean-engineering 29
department-of-chemical-engineering-and-materials-science 22
department-of-electrical-and-computer-engineering 35
school-of-humanities-arts-and-social-sciences 28
department-of-biomedical-engineering 11
department-of-mechanical-engineering 31
department-of-physics 17
graduate-education 24
schaefer-school-of-engineering-and-science 4
department-of-mathematical-sciences 14
academic-integrity 2
department-of-chemistry-and-chemical-biology 16
student-life 6
tuition-fees-and-other-expenses-for-undergraduate-students 5


In [6]:
sorted(primary["financing-a-stevens-education"])

['https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/financing-a-stevens-education/graduate-student-financial-information',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/financing-a-stevens-education/graduate-student-financial-information/financial-aid-sources',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/financing-a-stevens-education/graduate-student-financial-information/financial-aid-sources/fafsa',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/financing-a-stevens-education/graduate-student-financial-information/financial-aid-sources/federal-loans',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/financing-a-stevens-education/graduate-student-financial-information/financial-aid-sources/graduate-assistantships',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/financing-a-stevens-education/graduate-student-financial-information/financial-aid-sources/graduate-grant',
 'h

In [7]:
def create_department_school_metadata(url):
    metadata = {}
    path = urlparse(url).path.replace("/en/2023-2024/academic-catalog/", "")
    if "faculty" in path:
        metadata["content_type"] = "Faculty"
    elif "graduate-certificate" in path:
        metadata["content_type"] = "Certificate"
        metadata["program_type"] = "Certificate"
    elif "graduate" in path:
        metadata["content_type"] = "Graduate"
        if "master" in path:
            metadata["program_type"] = "Masters"
        elif "doctoral" in path:
            metadata["program_type"] = "PhD"
        elif "engineer" in path:
            metadata["program_type"] = "Engineer"
    elif "undergraduate" in path:
        metadata["content_type"] = "Undergraduate"
        metadata["program_type"] = "Bachelors"
    path = path.split("/")
    if len(path)>1:
        program_name = path[-1].replace("-", " ").title()
        program_name = program_name.replace("Copy Of", "").strip()
        program_name = program_name.replace("Master Of Science In", "").strip()
        program_name = program_name.replace("Bachelor Of Science In", "").strip()
        program_name = program_name.replace("Doctoral Program In", "").strip()
        program_name = program_name.replace("Engineering Degree In", "").strip()
        metadata["program_name"] = program_name
    return metadata

In [8]:
create_department_school_metadata(primary["department-of-computer-science"][0])

{'content_type': 'Faculty', 'program_name': 'Faculty'}

In [9]:
school_department_keys = [
    "school-of-business",
    "department-of-chemical-engineering-and-materials-science",
    "department-of-electrical-and-computer-engineering",
    "department-of-mathematical-sciences",
    "department-of-chemistry-and-chemical-biology",
    "department-of-computer-science",
    "department-of-civil-environmental-and-ocean-engineering",
    "department-of-mechanical-engineering",
    "department-of-physics",
    "school-of-systems-and-enterprises",
    "schaefer-school-of-engineering-and-science",
    "department-of-biomedical-engineering",
]

In [10]:
url_meta = {}
for key in primary.keys():
    if key in school_department_keys:
        for url in primary[key]:
            meta_data = create_department_school_metadata(url)
            meta_data["department"] = key.replace("-", " ").title()
            url_meta[url] = meta_data

In [11]:
len(url_meta.keys())

316

In [12]:
url_meta["https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/department-of-computer-science/graduate-programs/copy-of-master-of-science-in-computer-science"]

{'content_type': 'Graduate',
 'program_type': 'Masters',
 'program_name': 'Computer Science',
 'department': 'Department Of Computer Science'}

In [13]:
import pandas as pd
df = pd.read_csv("data/department_info.csv")

In [14]:
assert df.shape[0]==df["url"].str.endswith("/").sum()

In [15]:
df["url"] = df["url"].apply(lambda x: x[:-1])

In [16]:
df = df.set_index("url")

In [17]:
department_info = df.to_dict(orient="index")

In [18]:
department_info[list(url_meta.keys())[2]]

{'text': "\n\n\r\n\t\tBachelor of Science in Computer Science\r\n\tThe modern world revolves around continual advances in technology and creative programming solutions which require an innovative workforce with specialized programming skills. The modern world revolves around continual advances in technology and creative programming solutions which require an innovative workforce with specialized programming skills.\nProgram Description\nThe Bachelor of Science in Computer Science degree begins by immersing students in computer science-related courses in their first semester. Specialized computer science courses continue to follow, providing depth in many related issues and in a focus of your choosing;\xa0in the junior and senior years, a student can choose from elective courses within Computer Science, Mathematics, and several Engineering disciplines.\xa0Over the course of the program, students in this program take more computer science-related courses as part of their degree than at o

In [19]:
for url in url_meta:
    if department_info.get(url):
        url_meta[url].update({"page_content": department_info[url]["text"]})

In [20]:
url_meta_complete = []
for url, meta in url_meta.items():
    if meta.get("page_content"):
        # meta.update({"url": url})
        url_meta_complete.append(meta)

In [21]:
# for url in url_meta:
#     with open("stevens_scraper/department_urls.txt", "a") as fp:
#         fp.write(url+"\n")

In [22]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [23]:
model_kwargs = {
    "trust_remote_code": True,
    # "device": "cpu"
    }
encode_kwargs={"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [24]:
docs = [
    Document(
        page_content=doc["page_content"],
        metadata=dict((key, value) for key, value in doc.items() if key!="page_content")
    )
    for doc in url_meta_complete
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(docs, embeddings)

In [25]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="content_type",
        description="The type of the content for each department[Faculty, Certificate, Graduate, Undergraduate]",
        type="string",
    ),
    AttributeInfo(
        name="program_name",
        description="The name of the Program",
        type="string",
    ),
    AttributeInfo(
        name="department",
        description="The department or school that has that program",
        type="string",
    ),
    AttributeInfo(
        name="program_type",
        description="The type of the program offered by the university ['Masters', 'Bachelors', 'Certificate', 'PhD', 'Engineer']",
        type="string",
    ),
]
document_content_description = "URL of the content"
llm = ChatOpenAI(temperature=1)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True
)

In [26]:
question = """What are the elective courses for Masters of Science in Machine Learning?""".strip()

In [27]:
retriever_prompt = f"""
Pick the top 5 result for the following question: ```{question}```
""".strip()

In [28]:
# from langchain.chains.query_constructor.base import (
#     StructuredQueryOutputParser,
#     get_query_constructor_prompt,
# )
# prompt = get_query_constructor_prompt(
#     document_content_description,
#     metadata_field_info,
# )

# output_parser = StructuredQueryOutputParser.from_components()
# query_constructor = prompt | llm | output_parser

# print(prompt.format(query=retriever_prompt))

In [29]:
context = retriever.invoke(retriever_prompt)
context

[Document(page_content='\n\n\r\n\t\tMaster of Science in Machine Learning\r\n\tThe Master of Science in Machine Learning degree aims to provide theoretical and practical foundations that enable its graduates to be at the forefront of progress in machine learning and related disciplines. Machine learning is a rapidly expanding field with many applications in diverse areas, such as intelligent systems, perception, robotics, finance, information retrieval, bioinformatics, healthcare, weather prediction among others. In addition to established employers in these industries, a large number of startups have also entered the market seeking to hire machine learning scientists. Besides careers in industry, this program will prepare students for the pursuit of doctoral degrees and careers in research.\nProgram Objectives\n\xa0\nThe program prepares students to:\n\n\nEstablish a career as a machine learning scientist in either industry or academia.\n\n\nDevelop a deep understanding of machine lea

In [30]:
retriever

SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000016980105810>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Swift\\"), eq(\\"artist\\", \\"Katy Perry\\"))

In [31]:
context[0].metadata

{'content_type': 'Graduate',
 'department': 'Department Of Computer Science',
 'program_name': 'Machine Learning',
 'program_type': 'Masters'}

In [32]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [   
        ("system", "You are an assistant for question-answering tasks related to Stevens Institute Of Technology."),
        ("human", """ 
        Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.
        Answer in markdown format and render tables without code
        Question: {question}
        Context: {context}
        Answer:"""),
    ]
)
chat_template

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant for question-answering tasks related to Stevens Institute Of Technology.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=" \n        Use the following pieces of retrieved context to answer the question.\n        If you don't know the answer, just say that you don't know.\n        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.\n        Answer in markdown format and render tables without code\n        Question: {question}\n        Context: {context}\n        Answer:"))])

In [33]:
combine_docs_chain = create_stuff_documents_chain(llm, chat_template)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [34]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000016980105810>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{

In [70]:
response = retrieval_chain.invoke({"input": retriever_prompt, "question": question})
to_markdown(response["answer"])

> ### Elective Courses for Masters of Science in Machine Learning:
> 
> 
> | Course Code | Course Title                                |
> |-------------|---------------------------------------------|
> | CS 513      | Knowledge Discovery and Data Mining         |
> | CS 532      | 3D Computer Vision                          |
> | CS 544      | Health Informatics                          |
> | CS 556      | Mathematical Foundations of Machine Learning|
> | CS 558      | Computer Vision                             |
> | CS 582      | Causal Inference                            |
> | CS 589      | Text Mining and Information Retrieval       |
> | CS 598      | Visual Information Retrieval                |
> | CS 609      | Data Management and Exploration on the Web  |
> | BIA 654     | Experimental Design II                      |
> | BIA 660     | Web Mining                                  |
> | BIA 662     | Augmented Intelligence and Generative AI    |
> | BIA 678     | Big Data Technologies                       |
> | CPE 608     | Applied Modeling and Optimization           |
> | CPE 695     | Applied Machine Learning                    |
> | FE 541      | Applied Statistics with Applications in Finance|
> | MA 541      | Statistical Methods                         |
> | MA 630      | Advanced Optimization Methods               |
> | MA 641      | Time Series Analysis I                      |
> | MA 661      | Dynamic Programming and Reinforcement Learning |

In [71]:
vectorstore.delete_collection()

In [74]:
for key in primary.keys(): 
    if key not in school_department_keys:
        print(key, len(primary[key])) 

courses 2889
undergraduate-education 77
financing-a-stevens-education 44
student-services 26
 12
tuition-fees-and-other-expenses-for-undergraduate-students 5
graduate-education 24
school-of-humanities-arts-and-social-sciences 28
student-life 6
academic-integrity 2
tuition-fees-and-other-expenses-for-graduate-students 5
